In [16]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [17]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [51]:
import pandas as pd
import numpy as np

Import our white and red wine data from the csvs

# Read the CSV and Perform Basic Data Cleaning

In [52]:
red_wine_df = pd.read_csv("winequality-red.csv", sep=';')
white_wine_df = pd.read_csv("winequality-white.csv", sep=';')
white_wine_df.info()
white_wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64

## Balanced (Normalized) Data for White Wine Data (1599 data points) and Red Wine Data (1599 data points) 

In [53]:
#take a random sample of white wines to match the red wine data
random_white_df = white_wine_df.sample(1599,random_state=42).reset_index(drop=True)

Only balanced values are baseline condition we can use:

In [217]:
#if we want our only features of the model to have balanced white wine and red wine data, we use this data frame:
baseline_red_df = red_wine_df.copy()
baseline_white_df = random_white_df.copy()
only_balanced_df = pd.concat([baseline_red_df, baseline_white_df])
#only_balanced_df

In [58]:
balance_gp = only_balanced_df.groupby('quality')
balance_gp.quality.count()

quality
3      17
4      97
5    1146
6    1363
7     502
8      73
Name: quality, dtype: int64

In [59]:
only_balanced_df.groupby('quality').count()
# range_balanced_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,17,17,17,17,17,17,17,17,17,17,17
4,97,97,97,97,97,97,97,97,97,97,97
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
7,502,502,502,502,502,502,502,502,502,502,502
8,73,73,73,73,73,73,73,73,73,73,73


In [22]:
#Suppose we want to take specific ranges in only_balanced_df, which does not include colors
range_balanced_df = only_balanced_df.loc[\
(only_balanced_df["quality"] <= 7 ) & \
(only_balanced_df["quality"] >= 5 )  \
] 


In [23]:
range_balanced_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
7,502,502,502,502,502,502,502,502,502,502,502


In [220]:
#if we want to include wine colors as features in our model:
color_red_df = red_wine_df.copy()
color_white_df = random_white_df.copy()
color_red_df["color"] = "red"
color_white_df["color"] = "white"
colors_df = pd.concat([color_red_df, color_white_df])

Added Color with get dummies (Or in case we want to remove it)

In [25]:
#if we only want to use the balanced data and include the colors of wine:
dummy_colors_df = pd.get_dummies(colors_df, columns=['color'])
dummy_drop_colors_df = dummy_colors_df.drop(columns=["color_red"]).copy()
dummy_rename_colors_df = dummy_drop_colors_df.rename(columns={"color_white":"colors"})
dummy_rename_colors_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colors
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,5,1
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,5,1
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,6,1
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,5,1


If we want to hypertune further, and choose a specfic range of quality scores, so our model predicts a smaller range of quality scores

In [26]:
#For our model that includes our `colors` feature:
range_colors_df = dummy_rename_colors_df.loc[\
(dummy_rename_colors_df["quality"] <= 7 ) & \
(dummy_rename_colors_df["quality"] >= 5 )  \
] 


In [27]:
#To show which range we are using
range_colors_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors
quality,,,,,,,,,,,,
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
7,502,502,502,502,502,502,502,502,502,502,502,502


Add buckets to given range of quality values

In [28]:
only_balanced_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,17,17,17,17,17,17,17,17,17,17,17
4,97,97,97,97,97,97,97,97,97,97,97
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
7,502,502,502,502,502,502,502,502,502,502,502
8,73,73,73,73,73,73,73,73,73,73,73


### Moved quality values into buckets

Created bins for balanced data called: "terrible (0), mediocre (1), and great (2)"
The quality scores range from 0 to 4, 5 to 6, and 7 to 10. We will call them 0,1,2.

In [104]:
only_balanced_df.groupby('quality').count()
bin_balanced_df = only_balanced_df.copy()
bins = [0, 4, 6, 10]
group_names = [0, 1, 2]
bin_balanced_df["bin_quality"] = pd.cut(bin_balanced_df["quality"], bins, labels=group_names)
bin_qual_df = bin_balanced_df.drop(columns="quality")

bin_qual_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,bin_quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1


Now we want to add colors with the convert quality scores to bins to see what happens

In [110]:
bins_colors_df = dummy_rename_colors_df.copy()
bins = [0, 4, 6, 10]
group_names = [0, 1, 2]
bins_colors_df["bin_quality"] = pd.cut(bins_colors_df["quality"], bins, labels=group_names)
new_bins_colors_df = bins_colors_df.drop(columns="quality")
new_bins_colors_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors,bin_quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,0,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,1,1
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,1,1
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,1,1
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,1,1


In [90]:
# balance_dum_df = pd.get_dummies(bin_qual_df, columns=['bin_quality'])
# balance_dum_df.sample(50)

# Unbalanced (Denormalized) Data for White Wine Data (4898 data points) and Red Wine Data (1599 data points) 

# Select your features (columns)

In [225]:
# Set features. Drop quality, y value. 
X = new_bins_colors_df.drop(columns='bin_quality')
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


# Create a Train Test Split

Use `quality` for the y values

In [226]:
from sklearn.model_selection import train_test_split
#80% train, 20% test. y is this one column
y = new_bins_colors_df['bin_quality']
#random state 42 will have same picks for x test and y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [227]:
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors
1251,7.5,0.58,0.14,2.2,0.077,27.0,60.0,0.99630,3.28,0.59,9.8,0
1080,10.3,0.27,0.56,1.4,0.047,3.0,8.0,0.99471,3.16,0.51,11.8,0
272,7.2,0.19,0.46,3.8,0.041,82.0,187.0,0.99320,3.19,0.60,11.2,1
1187,7.0,0.43,0.30,2.0,0.085,6.0,39.0,0.99346,3.33,0.46,11.9,0
212,11.6,0.44,0.64,2.1,0.059,5.0,15.0,0.99800,3.21,0.67,10.2,0


# Train the Model using Random Forest 



In [228]:
#Train the model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [229]:
#Test the model
from sklearn.metrics import confusion_matrix, classification_report
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.86      0.95      0.90       632
           2       0.67      0.48      0.56       145

    accuracy                           0.83       800
   macro avg       0.51      0.48      0.49       800
weighted avg       0.80      0.83      0.81       800



/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'rfcredwhitewine567.sav'
joblib.dump(predictions, filename)